<a href="https://colab.research.google.com/github/oneextrafact/RESDSQL/blob/main/daveg_resdsql_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create directories

In [2]:
!git clone https://github.com/oneextrafact/RESDSQL

Cloning into 'RESDSQL'...
remote: Enumerating objects: 221, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 221 (delta 37), reused 29 (delta 29), pack-reused 171
Receiving objects: 100% (221/221), 2.33 MiB | 9.49 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [2]:
!mv RESDSQL/* ./

In [3]:
!mkdir third_party
!mkdir eval_results
!mkdir models
!mkdir tensorboard_log
!mkdir third_party
!mkdir predictions

mkdir: cannot create directory ‘third_party’: File exists


Clone additional repos

In [4]:
!git clone https://github.com/ElementAI/spider.git
!git clone https://github.com/ElementAI/test-suite-sql-eval.git
!mv spider third_party/
!mv test-suite-sql-eval third_party/test-suite

Cloning into 'spider'...
remote: Enumerating objects: 454, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 454 (delta 15), reused 11 (delta 11), pack-reused 422
Receiving objects: 100% (454/454), 43.82 MiB | 23.52 MiB/s, done.
Resolving deltas: 100% (151/151), done.
Cloning into 'test-suite-sql-eval'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 115 (delta 27), reused 20 (delta 20), pack-reused 80
Receiving objects: 100% (115/115), 634.79 KiB | 3.22 MiB/s, done.
Resolving deltas: 100% (59/59), done.


My objective is to see if it's possible to fine tune

Download training data and databases from Spider set

In [5]:
!gdown 19tsgBGAxpagULSl9r85IFKIZb4kyBGGu
!gdown 1s4ItreFlTa8rUdzwVRmUR2Q9AHnxbNjo

Downloading...
From: https://drive.google.com/uc?id=19tsgBGAxpagULSl9r85IFKIZb4kyBGGu
To: /content/data.zip
100% 669M/669M [00:07<00:00, 86.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s4ItreFlTa8rUdzwVRmUR2Q9AHnxbNjo
To: /content/database.zip
100% 98.2M/98.2M [00:01<00:00, 58.7MB/s]


Download Models

In [6]:
!gdown 1UWNj1ZADfKa1G5I4gBYCJeEQO6piMg4G
!gdown 1zHAhECq1uGPR9Rt1EDsTai1LbRx0jYIo
!gdown 1QyfSfHHrxfIM5X9gKUYNr_0ZRVvb1suV
!gdown 1lqZ81f_fSZtg6BRcRw1-Ol-RJCcKRsmH


Downloading...
From: https://drive.google.com/uc?id=1UWNj1ZADfKa1G5I4gBYCJeEQO6piMg4G
To: /content/text2natsql_schema_item_classifier.zip
100% 1.38G/1.38G [00:09<00:00, 148MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1zHAhECq1uGPR9Rt1EDsTai1LbRx0jYIo
To: /content/text2sql_schema_item_classifier.zip
100% 1.38G/1.38G [00:14<00:00, 94.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QyfSfHHrxfIM5X9gKUYNr_0ZRVvb1suV
To: /content/text2natsql-t5-base.zip
100% 829M/829M [00:09<00:00, 89.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lqZ81f_fSZtg6BRcRw1-Ol-RJCcKRsmH
To: /content/text2sql-t5-base.zip
100% 829M/829M [00:06<00:00, 121MB/s]


Unzip data and database

In [7]:
!unzip data.zip
!unzip database.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/car_1/schema_0.sql  
 extracting: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/car_1/car_1.  
   creating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/orchestra/
  inflating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/orchestra/schema.sql  
  inflating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/orchestra/orchestra.sqlite  
  inflating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/orchestra/schema_0.sql  
   creating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/employee_hire_evaluation/
  inflating: data/diagnostic-robustness-text-to-sql/data/NLQ_column_synonym/databases/employee_hire_evaluation/employee_hire_evaluation.sqlite  
  inflating: data/diagnostic-robustness-text-to-sql/

Unzip weights and models

In [8]:
!unzip text2natsql-t5-base.zip -d models/
!unzip text2natsql_schema_item_classifier.zip -d models/
!unzip text2sql-t5-base.zip -d models/
!unzip text2sql_schema_item_classifier.zip -d models/

Archive:  text2natsql-t5-base.zip
   creating: models/text2natsql-t5-base/
   creating: models/text2natsql-t5-base/checkpoint-14352/
  inflating: models/text2natsql-t5-base/checkpoint-14352/added_tokens.json  
  inflating: models/text2natsql-t5-base/checkpoint-14352/tokenizer_config.json  
  inflating: models/text2natsql-t5-base/checkpoint-14352/special_tokens_map.json  
  inflating: models/text2natsql-t5-base/checkpoint-14352/config.json  
  inflating: models/text2natsql-t5-base/checkpoint-14352/tokenizer.json  
  inflating: models/text2natsql-t5-base/checkpoint-14352/spiece.model  
  inflating: models/text2natsql-t5-base/checkpoint-14352/pytorch_model.bin  
Archive:  text2natsql_schema_item_classifier.zip
   creating: models/text2natsql_schema_item_classifier/
  inflating: models/text2natsql_schema_item_classifier/added_tokens.json  
  inflating: models/__MACOSX/text2natsql_schema_item_classifier/._added_tokens.json  
  inflating: models/text2natsql_schema_item_classifier/tokenizer_c

To reproduce this, I had to remove versions from Spacy and tensorboard, then set protobuf to 3.20.0. Also had to reset the runtime after running the install, not sure why.

In [9]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 13.3 MB/s eta 0:00:00
   

Running inference will error out if you don't call punkt from nltk.

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Unfortunately, it seems that this is too big for the regular GPU, even for inference. It is just massive and maybe not economic to run without a dedicated GPU machine. Who's going to buy that kind of thing?

In [1]:
!sh scripts/inference/infer_text2sql.sh base spider

sh: 0: Can't open scripts/inference/infer_text2sql.sh


The real question is, then, how do I actually perform inference? I've got the model, so how do I wrap that part in a function? for that, I need to inspect the actual inference function.

Step 2 would be fine tuning, or even training just on our own set... I need a database, data, and queries for that. That's what all of us need, really, what will really help the guys take off.

In [1]:
!spacy download en_core_web_sm

2023-07-02 20:14:21.841564: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-02 20:14:21.894975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 88.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
%cd RESDSQL/

/content/RESDSQL


In [5]:
!mv ../data/ .

In [8]:
!sh scripts/inference/infer_text2sql.sh base spider

0it [00:00, ?it/s]
Traceback (most recent call last):
  File "/content/RESDSQL/preprocessing.py", line 410, in <module>
    main(opt)
  File "/content/RESDSQL/preprocessing.py", line 354, in main
    db_contents = get_db_contents(
  File "/content/RESDSQL/preprocessing.py", line 45, in get_db_contents
    matches = get_database_matches(
  File "/content/RESDSQL/utils/bridge_content_encoder.py", line 234, in get_database_matches
    picklist = get_column_picklist(
  File "/content/RESDSQL/utils/bridge_content_encoder.py", line 222, in get_column_picklist
    conn.close()
UnboundLocalError: local variable 'conn' referenced before assignment


In [12]:
!mv text2sql_schema_item_classifier/ models/

In [13]:
!rm -rf third_party_doop/

In [7]:
!pip install --upgrade protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.0
    Uninstalling protobuf-3.19.0:
      Successfully uninstalled protobuf-3.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.23.3 which is incompatible.
tensorflow 2.12.0 requires tensorboard<2.13,>=2.12, but you have tensorboard 2.11.2 which is incompatible.
